# This Jupyter notebook generates a network graph file for keyword and hashtag relations to each others

In [1]:
import pandas as pd

##Processed csv file from another notebook
filename=""
df = pd.read_csv(filename,low_memory=False)
df['combine_text'] = df['combine_text'].str.lower()
df = df[~df['combine_text'].str.contains('huhu', na=False)]


In [2]:

disinformaatio=df[df['combine_text'].str.contains('disinformaatio', regex=False)]
disinformaatio['text']=disinformaatio['combine_text'].map(lambda s: [i  for i in s.split() if i.startswith("disinformaatio") ])
disinformaatio['keypair'] = "disinformaatio"

misinformaatio=df[df['combine_text'].str.contains('misinformaatio', regex=False)]
misinformaatio['text']=misinformaatio['combine_text'].map(lambda s: [i  for i in s.split() if i.startswith("misinformaatio") ])
misinformaatio['keypair'] = "misinformaatio"

propaganda=df[df['combine_text'].str.contains('propaganda', regex=False)]
propaganda['text']=propaganda['combine_text'].map(lambda s: [i  for i in s.split() if i.startswith("propaganda") ])
propaganda['keypair'] = "propaganda"

uutisankka=df[df['combine_text'].str.contains('uutisankka', regex=False)]
uutisankka['text']=uutisankka['combine_text'].map(lambda s: [i  for i in s.split() if i.startswith("uutisankka") ])
uutisankka['keypair'] = "uutisankka"

valeuutinen=df[df['combine_text'].str.contains('valeuutinen ', regex=False)]
valeuutinen['text']=valeuutinen['combine_text'].map(lambda s: [i  for i in s.split() if i.startswith("valeuutinen") ])
valeuutinen['keypair'] = "valeuutinen"

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [3]:
twitterData=pd.concat([disinformaatio, misinformaatio, propaganda, uutisankka, valeuutinen], ignore_index=True)

In [4]:
import json
array2=[]
array=[]
dataframe_obj={}

df2= pd.DataFrame({'text': [], 'hashtag': [], 'tweet': [], 'keyword': []})
for index in twitterData.index:
    array=[]
    dataframe_obj={}
    item = twitterData["combine_hastags"][index]
    textdata = twitterData["text"][index]
    if item != '[]':
        if textdata != []:
            test=json.loads(item)
            tag=[]
            for x in test:
                tag.append(x['text'].lower())
            df2 = df2.append({'text': twitterData["text"][index][0], 'hashtag':  tag, 'tweet': "tweet"+str(index),'keyword': twitterData["keypair"][index]}, ignore_index=True)

df2        

,text,hashtag,tweet,keyword
0,disinformaatiosta.,"[disinformation, valeuutinen]",tweet1,disinformaatio
1,disinformaatio,[ilmastokriisi],tweet6,disinformaatio
2,disinformaatio,[ilmastokriisi],tweet7,disinformaatio
3,disinformaatio,[ilmastokriisi],tweet8,disinformaatio
4,disinformaatio,[ilmastokriisi],tweet9,disinformaatio
...,...,...,...,...
779,valeuutinen,[trumpulos],tweet8637,valeuutinen
780,valeuutinen,"[johanbäckman, vaalivilppi, valeuutiset, vaali...",tweet8696,valeuutinen
781,valeuutinen,"[valeuutinen, fakenews, fakemedia, valemedia]",tweet8713,valeuutinen
782,valeuutinen,"[valeuutinen, fakenews, medialukutaito]",tweet8717,valeuutinen


In [5]:
#from pylab import rcParams
import networkx as nx
import nxviz as nv
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.use('tkagg')    #YAAA!!  this finally makes the Damn thing work
import matplotlib.pyplot as plt
#%matplotlib inline

user_to_hashtags=dict(df2[['keyword','hashtag']].values) #a more convenient data structure: a dictionary with keyword as keys and the list of hashtags they use as values.
B=nx.DiGraph() #create an empty graph
for i in df2.index: #loop over all the keywords
    for hashtag in df2.hashtag[i]: #for each keyword loop over the hashtags they use
        if not B.has_edge(df2.keyword[i], hashtag): #if no weight then add 0
            B.add_edge(df2.keyword[i],hashtag,weight=0) #add the edge User<->hashtag
        B[df2.keyword[i]][hashtag]['weight'] += 1 #if has weight add more wieght to hashtag showing again.

nx.readwrite.gexf.write_gexf(B,
  'tags_network.gexf',
   encoding='utf-8', version='1.2draft')    

"\nnx.readwrite.gexf.write_gexf(B,\n  'tags_network.gexf',\n   encoding='utf-8', version='1.2draft')    \n   "

In [6]:
# Compute the betweenness centrality of G: bet_cen
bet_cen = nx.betweenness_centrality(B)
# Compute the degree centrality of G: deg_cen
deg_cen = nx.degree_centrality(B)
# Compute the page rank of G: page_rank
page_rank = nx.pagerank(B)
# Compute the closeness centrality of G: clos_cen
clos_cen = nx.closeness_centrality(B)
sorted(bet_cen.items(), key=lambda x:x[1], reverse=True)[0:5]

[('propaganda', 0.002692775257839767),
 ('disinformaatio', 0.0019727410970662696),
 ('valeuutinen', 0.00020805828246253866),
 ('misinformaatio', 9.368069262711163e-05),
 ('disinformation', 0.0)]

In [16]:
# How to get the number of nodes
print(len(B.nodes()))
# How to get the number of edges
print(len(B.edges()))

679
804


In [12]:
nx.average_shortest_path_length(B)

0.010706791611818525

In [13]:
nx.density(B)

0.0017464517053970572

In [14]:
nx.average_clustering(B)

0.09943887007744538

In [15]:
nx.transitivity(B)

0.0008594353997312857